In [0]:
import pandas as pd
from sklearn.preprocessing import scale
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Get Training and Validation Sets

Here we gather a random sample of 2/3rds of the data to create both a training and validation set.

In [0]:
train_set = pd.read_csv('gdrive/My Drive/sbgan/training_data.csv', index_col=0).reset_index(drop=True)
val_set = pd.read_csv('gdrive/My Drive/sbgan/validation_data.csv', index_col=0).reset_index(drop=True)

# Extract Appropriate Data

Extract the fields we want from training and validation sets. We also need to get the odds to calculate profitability in the validation data.

In [0]:
X_train = train_set.drop(['ODDS', 'OUTCOME'], axis=1)
y_train = train_set['OUTCOME']

X_val = val_set.drop(['ODDS', 'OUTCOME'], axis=1)
y_val = val_set['OUTCOME']

odds = val_set['ODDS']

X_train

,WIN%_WINNER,PTS_WINNER,FGM_WINNER,FGA_WINNER,FG%_WINNER,3PM_WINNER,3PA_WINNER,3P%_WINNER,FTM_WINNER,FTA_WINNER,FT%_WINNER,OREB_WINNER,DREB_WINNER,AST_WINNER,TOV_WINNER,STL_WINNER,BLK_WINNER,BLKA_WINNER,PF_WINNER,PFD_WINNER,+/-_WINNER,OFFRTG_WINNER,DEFRTG_WINNER,NETRTG_WINNER,AST%_WINNER,AST/TO_WINNER,ASTRATIO_WINNER,OREB%_WINNER,DREB%_WINNER,REB%_WINNER,TOV%_WINNER,EFG%_WINNER,TS%_WINNER,PACE_WINNER,PIE_WINNER,FTARATE_WINNER,OPPEFG%_WINNER,OPPFTARATE_WINNER,OPPTOV%_WINNER,OPPOREB%_WINNER,...,OPPPITP_LOSER,%FGA2PT_LOSER,%FGA3PT_LOSER,%PTS2PT_LOSER,%PTS2PT-MR_LOSER,%PTS3PT_LOSER,%PTSFBPS_LOSER,%PTSFT_LOSER,%PTSOFFTO_LOSER,%PTSPITP_LOSER,2FGM%AST_LOSER,2FGM%UAST_LOSER,3FGM%AST_LOSER,3FGM%UAST_LOSER,FGM%AST_LOSER,FGM%UAST_LOSER,OPPFGM_LOSER,OPPFGA_LOSER,OPPFG%_LOSER,OPP3PM_LOSER,OPP3PA_LOSER,OPP3P%_LOSER,OPPFTM_LOSER,OPPFTA_LOSER,OPPFT%_LOSER,OPPOREB_LOSER,OPPDREB_LOSER,OPPREB_LOSER,OPPAST_LOSER,OPPTOV_LOSER,OPPSTL_LOSER,OPPBLK_LOSER,OPPBLKA_LOSER,OPPPF_LOSER,OPPPFD_LOSER,OPPPTS_LOSER,OPP+/-_LOSER,OPPPTS2NDCHANCE_LOSER,OPPPTSFB_LOSER,OPPPTSPAINT_LOSER
0,0.444,-0.977860,-1.536727,-0.845358,0.429,-0.482446,0.016104,0.300,0.807257,0.666005,0.772,0.112221,-0.612435,-1.401929,1.384743,1.782603,-0.766028,0.187384,0.819961,1.279438,0.132884,-1.389504,-1.709050,0.140708,0.547,-1.782645,-1.512716,0.296,0.714,0.499,0.172,0.471,0.521,-0.264547,0.055776,0.869978,0.486,0.895727,0.182,0.286,...,-0.688851,0.752,0.248,0.582,0.164,0.204,0.108,0.214,0.161,0.418,0.463,0.537,0.831,0.169,0.533,0.467,-0.629332,-0.015150,0.436,-1.016631,-0.998663,0.351,-0.787086,-0.651989,0.745,0.584581,-0.377488,-0.094579,-0.771560,0.102330,1.236210,1.092759,-1.115536,1.160450,-0.937112,-1.055549,-0.238085,-0.287141,1.566566,-0.688851
1,0.200,0.701097,0.949025,0.994177,0.459,0.714700,1.049039,0.333,-0.723071,-0.897560,0.776,-1.340156,-0.492926,0.869716,0.910959,1.478103,0.137509,-0.175947,1.867855,-0.525978,-1.828037,-0.291588,2.131022,-1.695006,0.597,0.033760,0.508716,0.247,0.687,0.465,0.155,0.515,0.551,1.420324,-1.354220,-1.076329,0.544,0.681684,0.156,0.313,...,1.047926,0.607,0.393,0.509,0.065,0.318,0.087,0.174,0.155,0.444,0.549,0.451,0.842,0.158,0.635,0.365,0.769692,0.928952,0.455,1.044930,0.990900,0.363,-0.640025,-0.613616,0.756,-1.839417,0.868134,-0.008498,-0.465953,-0.048169,1.236210,-0.208563,0.746932,1.160450,0.165235,0.703066,-0.545638,-1.281381,-1.245284,1.047926
2,0.677,1.271018,1.481686,0.807558,0.476,0.012924,0.046938,0.353,0.760883,0.447833,0.789,1.491980,1.140365,0.335211,0.753031,-0.348897,1.379871,-0.418167,-0.166291,-0.525978,1.245330,1.554907,-0.167613,1.283346,0.551,-0.218518,-0.035516,0.339,0.716,0.535,0.158,0.523,0.565,0.564301,1.199734,0.065505,0.487,-0.415285,0.134,0.284,...,0.072476,0.808,0.192,0.656,0.153,0.169,0.138,0.175,0.189,0.503,0.552,0.448,0.899,0.101,0.603,0.397,-0.192137,-0.066182,0.449,0.642675,0.674801,0.357,0.536463,0.537583,0.761,0.957504,-0.288515,0.163664,1.011146,0.553829,0.999838,0.501249,0.863337,-0.584437,0.471442,0.255419,0.723019,0.860058,0.264784,0.072476
3,0.600,1.579083,1.614851,1.820635,0.459,2.118251,2.051141,0.367,-1.001313,-1.406627,0.797,-0.396111,1.100529,1.849640,-1.063141,0.970603,0.589277,-1.387048,0.080272,-0.525978,1.113345,1.330333,-0.167613,1.133492,0.629,2.354722,1.519432,0.266,0.733,0.500,0.131,0.533,0.566,1.230096,1.040112,-1.673197,0.506,-0.227998,0.155,0.267,...,1.571338,0.619,0.381,0.526,0.051,0.316,0.151,0.157,0.167,0.476,0.546,0.454,0.837,0.163,0.629,0.371,1.993838,1.541341,0.473,2.050570,1.976384,0.364,1.467849,1.765529,0.744,-0.254495,1.179539,0.981431,2.793852,1.155826,4.072673,0.382947,0.514124,1.289701,1.879996,2.573593,1.491902,1.089498,2.451778,1.571338
4,0.694,1.933359,1.170967,0.860878,0.468,2.696183,2.621568,0.367,0.343521,0.338747,0.760,-0.033016,0.343638,1.671472,0.674067,0.564603,-0.314260,-0.054836,-0.659418,-0.525978,1.151055,2.228628,0.670713,1.208419,0.633,0.790596,1.441685,0.288,0.713,0.502,0.155,0.552,0.586,1.023564,0.694264,-0.012348,0.517,-0.736350,0.150,0.287,...,0.001101,0.64

# Define Profitability Metric

We need a custom Keras `Callback` to track Profitability from Epoch to Epoch. This is a bit hacky so there might be a better solution.

Profitability is calculated from the odds, wager, and outcome. Odds are in decimal format. Wager is calculated using Kelly's criterion for betting.

Kelly's criterion is:

```f* = (b*p - q) / b```

Where...
* f* is the percentage bet
* b is `odds - 1`
* p is probability of victory
* q is `1 - p`

The great thing about Kelly is that if the probability of victory is low enough, the bet will be negative. Therefore, when we return a wager, we just need to return `max(0, kelly)`. 

In this formula, we multiply `kelly` by `100` to represent "units." A single unit is `1%` of your bankroll. This makes sense to people who bet.

Additionally, some bets will be huge. We want to limit this so we'll limit bets to `15`, although it might be useful to set this higher. 

In [0]:
import numpy as np
from scipy.stats import zscore
import tensorflow.keras
from tensorflow.keras.callbacks import Callback
import cvxopt as opt  
from cvxopt import blas, solvers

solvers.options['show_progress'] = False

# https://datascience.stackexchange.com/questions/13746/how-to-define-a-custom-performance-metric-in-keras
class Profit(Callback):
  def on_train_begin(self, logs={}):
    self.fixed = []
    self.kelly = []
  
  # Bet a fixed rate
  def calculate_fixed(self, wager, predictions, odds, targets):
    data = zip(predictions, odds, targets)
    profits = []
    wagers = []
    for pred, bprime, outcome in data:
      prof = 0
      wagers.append(wager)
      if pred == 1:
        prof = wager * bprime if outcome == 1 else 0
      profits.append(prof)
    self.fixed.append(np.sum(profits) - np.sum(wagers))
  
  # Bet based on Kelly's criterion
  def calculate_kelly(self, scores, odds, targets, max_wager=15):
    data = zip(scores, odds, targets)
    profits = []
    wagers = []
    for p, bprime, outcome in data:
      b = bprime - 1
      q = 1 - p
      units = max(0, round(float(((b*p - q) / b) * max_wager)))
      wagers.append(units)
      prof = units * bprime if outcome == 1 else 0
      profits.append(prof)
    self.kelly.append(np.sum(profits) - np.sum(wagers))

  def on_epoch_end(self, epoch, logs={}):
    scores = np.asarray(self.model.predict(self.validation_data[0]))
    predictions = np.round(scores)
    targets = self.validation_data[1]
    odds = np.asarray(self.validation_data[2].values)
    self.calculate_fixed(5, predictions, odds, targets)
    self.calculate_kelly(scores, odds, targets)

# Define and Train Model

Model is a Dense Feed Forward Binary Classification network. Output is the probability that the proposed winner will actually win.

It's kind of hard to understand what this means so basically:

1. Every game has two lines
2. Each point represents a line formatted like: `(proposed winner stats, proposed loser stats, odds)`
3. Network will output probability that `proposed winner` ACTUALLY wins

For some reason sigmoid works the best here. I'll need to do some more legitimate testing to find out if I can tune the model more. It seems like the accuracy limit is 67%.

TODO: Try to hyperparameterize a little bit.

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, Callback, ModelCheckpoint

model = Sequential()
model.add(Dense(128, input_dim=174, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

stop = EarlyStopping(monitor='val_loss', patience=50, verbose=1)
profit = Profit()
checkpoint = ModelCheckpoint('gdrive/My Drive/sbgan/models/binary.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

history = model.fit(X_train, 
                    y_train, 
                    validation_data=(X_val, y_val, odds), 
                    epochs=40000, 
                    callbacks=[checkpoint, stop], 
                    batch_size=128)

Epoch 1/40000
138/138 [==============================] - 1s 5ms/step - loss: 0.6646 - accuracy: 0.5998 - val_loss: 1.5569 - val_accuracy: 0.6411
Epoch 2/40000
138/138 [==============================] - 1s 4ms/step - loss: 0.6348 - accuracy: 0.6505 - val_loss: 1.5677 - val_accuracy: 0.6462
Epoch 3/40000
138/138 [==============================] - 1s 4ms/step - loss: 0.6328 - accuracy: 0.6531 - val_loss: 1.5464 - val_accuracy: 0.6451
Epoch 4/40000
138/138 [==============================] - 1s 4ms/step - loss: 0.6281 - accuracy: 0.6527 - val_loss: 1.5361 - val_accuracy: 0.6488
Epoch 5/40000
138/138 [==============================] - 1s 4ms/step - loss: 0.6264 - accuracy: 0.6523 - val_loss: 1.5317 - val_accuracy: 0.6454
Epoch 6/40000
138/138 [==============================] - 1s 4ms/step - loss: 0.6254 - accuracy: 0.6561 - val_loss: 1.5272 - val_accuracy: 0.6454
Epoch 7/40000
138/138 [==============================] - 1s 4ms/step - loss: 0.6234 - accuracy: 0.6553 - val_loss: 1.5392 - val_ac

In [0]:
model.save('gdrive/My Drive/sbgan/binary')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: gdrive/My Drive/sbgan/binary/assets


# Analyze Profitability

In [0]:
print(f"Kelly: {np.average(profit.kelly)}\n")
print(f"Fixed: {np.average(profit.fixed)}\n")

# Predict Games


In [0]:
def predict_wagers(games, odds, model):
  keys = games['LINE']
  games = games.drop(['WINS_WINNER', 'LOSSES_WINNER', 'WINS_LOSER', 'LOSSES_LOSER', 'LINE', 'ODDS'], axis=1)
  scores = model.predict(games)

  data = zip(keys, scores, odds)

  predictions = {}

  for k, p, bprime in data:
    b = bprime - 1
    q = 1 - p
    units = round(float((((b*p) - q) / b) * 15))
    predictions[k] = max(0, units)
    
  return predictions

def predict_winners(games, model):
  keys = games['LINE']
  games = games.drop(['WINS_WINNER', 'LOSSES_WINNER', 'WINS_LOSER', 'LOSSES_LOSER', 'LINE', 'ODDS'], axis=1)
  scores = model.predict(games)

  data = zip(keys, scores)

  predictions = {}

  for k, p in data:
    predictions[k] = p
  return predictions

In [0]:
from keras.models import load_model
from google.colab import files
model = load_model('gdrive/My Drive/sbgan/models/binary.h5')
games = pd.read_csv('gdrive/My Drive/sbgan/nightly/24FEB.csv', index_col=0)

odds = games['ODDS']

winners = predict_winners(games, model)
wagers = predict_wagers(games, odds, model)

print(winners)
print(wagers)